# SplitOut: Out-of-the-Box Training-Hijacking Detection in Split Learning via Outlier Detection

In [ ]:
import numpy as np
import torch
import random
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.utils import save_image
from scipy.stats import sem
import math
import itertools
import statistics
import pickle
# import architectures_torch as architectures

from models import *
from util import *

from tqdm.notebook import tqdm
from torchvision.models import resnet18

import matplotlib.pyplot as plt
import pandas as pd
import time
import io

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Running on', device)

In [ ]:
trainloader, testloader = load_dataset('cifar')
print(len(trainloader))

In [ ]:
TOTAL_BATCHES = len(trainloader)
# TOTAL_BATCHES = 3750
TOTAL_BATCHES

In [ ]:
# Colab Drive Connection
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# initialization of pickle loader
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
          return super().find_class(module, name)

### 1) Loading FSHA and honest gradients

In [7]:
def load_gradients(file_name, number_of_files, print_info=True):
    grads_original = []

    for pickled_file in range(number_of_files):
        file = open(file_name + str(pickled_file), 'rb')

        if torch.cuda.is_available():
            gradients_list = pickle.load(file)
        else:
            gradients_list = CPU_Unpickler(file).load()

        grads_original.append(gradients_list)
        file.close()

    if print_info:
        print("Data structure of a gradient: ",type(grads_original))
        print("Gradients brief info: {} epochs, {} gradients in an epoch, single gradient length in size  {}".format(
            len(grads_original), len(grads_original[0]), len(grads_original[0][0])
        ))
        print()

    return grads_original

In [ ]:
drive_path = '/content/drive/MyDrive/grads'
# load all gradients from honest server
honest_grads_original =  load_gradients(drive_path+'/HONEST_cifar10/honest_cifar_grads_', 100)

In [ ]:
# load all gradients from FSHA server
fsha_grads_original =  load_gradients(drive_path+'/SSPY_cifar10/FSHA_newAtt_cifar10_regulars_', 100)

### 2) Client NN training

In [ ]:
torch.cuda.empty_cache()
REPS = 100 # number of reps to average scores from
model_str = 'resnet'
optimizer = 'adam'
dataset = 'cifar'
del trainloader, testloader
trainloader, testloader = load_dataset(dataset)
print(dataset, model_str)

# other params
NUM_CLASSES = 100 if dataset == 'cifar100' else 10
EPOCHS = 1 # number of epochs to run simulation for
SETUPS = [('honest')]

client_collected_grads = []

for rep in tqdm(range(REPS)):
    client_temp_collected_grads = []
    start_time_nn_train = time.time()

    for adv_type in SETUPS:
        model = get_models(model_str,  dataset, device)

        client_opt, server_opt = get_optims(optimizer, model, model)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(1):
            for index, item in enumerate(tqdm(trainloader, leave=False)):
                images, labels = item[0].to(device), item[1].to(device)
                client_opt.zero_grad()
                server_opt.zero_grad()

                pred = model(images)
                loss = criterion(pred, labels)

                # loss backward, collect client grad
                loss.backward()
                client_grad = list(model.parameters())[0].grad.detach().clone().flatten()
                # # client_temp_collected_grads.append(client_grad) # collect train grads

    end_time_nn_train = time.time()
    print(f"Setup {rep} training is completed. Elasped time: {round(end_time_nn_train-start_time_nn_train,3)}")
    # print("len collected train grads: ", len(cli_10_collected_grads))
    client_collected_grads.append(client_temp_collected_grads)


print("Number of epochs: ", len(client_collected_grads))

torch.cuda.empty_cache()


In [ ]:
for i in range(len(client_collected_grads)):
  print(len(client_collected_grads[i]), "==>", len(client_collected_grads[i][0]))

### 3) Converting Tensor gradients to NumPy Array


In [13]:
def grad_tensor_to_numpy_converter(original_gradients, print_info=True):
    new_gradients_nparray = []

    for new_epoch in range(len(original_gradients)):

        new_grads_one_epoch = []
        for new_gradient in range(len(original_gradients[new_epoch])):
            new_grads_one_epoch.append(original_gradients[new_epoch][new_gradient].detach().cpu().numpy())
        new_gradients_nparray.append(new_grads_one_epoch)

    if print_info == True:
        print("Initial D.S. of one gradient in an epoch: ",type(original_gradients[0][0]))
        print("After conversion, D.S. of one gradient in an epoch: ",type(new_gradients_nparray[0][0]))
        print("Gradients brief info: {} epochs, {} gradients, single gradient length in size  {}".format(
            len(new_gradients_nparray), len(new_gradients_nparray[0]), len(new_gradients_nparray[0][0])
        ))
        print()

    return new_gradients_nparray


In [14]:
honest_grads_nparray = grad_tensor_to_numpy_converter(honest_grads_original)
fsha_grads_nparray = grad_tensor_to_numpy_converter(fsha_grads_original)

Initial D.S. of one gradient in an epoch:  <class 'torch.Tensor'>
After conversion, D.S. of one gradient in an epoch:  <class 'numpy.ndarray'>
Gradients brief info: 100 epochs, 782 gradients, single gradient length in size  1728

Initial D.S. of one gradient in an epoch:  <class 'torch.Tensor'>
After conversion, D.S. of one gradient in an epoch:  <class 'numpy.ndarray'>
Gradients brief info: 100 epochs, 809 gradients, single gradient length in size  1728



### 3.1. (optional) Reduce number of batches
If your number of iterations is more than the required number of batches, you can
reduce the number of batches:

In [16]:
def grads_list_epoch_reducer(grads_nparray, TOTAL_BATCHES):
  grads_nparray_REDUCED = []

  CUT_GRAD = TOTAL_BATCHES
  print(f"Cut gradients at: {CUT_GRAD}")

  for epoch_in_arr in range(len(grads_nparray)):

    grads_one_epoch = []

    if len(grads_nparray[epoch_in_arr]) < CUT_GRAD:
      grads_one_epoch = grads_nparray[epoch_in_arr]
    else:
      for grad in range(CUT_GRAD):
          grads_one_epoch.append(grads_nparray[epoch_in_arr][grad])

    grads_nparray_REDUCED.append(grads_one_epoch)

  print("Number of honest epochs after reduced data rate: {}".format(len(grads_nparray_REDUCED)))

  # check number of gradients and rates for first 10 epoch
  print("First 10 epoch number of gradients and data rates:")
  for i in range(10):
    print("({}, {}%) ".format(
      len(grads_nparray_REDUCED[i]),
      round((len(grads_nparray_REDUCED[i])/len(grads_nparray_REDUCED[i])),2)*100),
      end= " ")
  return grads_nparray_REDUCED

In [17]:
fsha_grads_nparray_REDUCED = grads_list_epoch_reducer(fsha_grads_nparray, TOTAL_BATCHES)

Cut gradients at: 782
Number of honest epochs after reduced data rate: 100
First 10 epoch number of gradients and data rates:
(782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  

In [18]:
honest_grads_nparray_REDUCED = grads_list_epoch_reducer(honest_grads_nparray, TOTAL_BATCHES)


Cut gradients at: 782
Number of honest epochs after reduced data rate: 100
First 10 epoch number of gradients and data rates:
(782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  (782, 100.0%)  

### 4) Selecting desired reduced data rate

In [55]:
honest_grads_reduced_data_rate = []

# random percentage selection method
data_rate_honest_gradients = 1
CUT_GRAD = int(len(honest_grads_nparray_REDUCED[0])* (data_rate_honest_gradients/100))
print(f"Cut gradients at: {CUT_GRAD}")

for epoch_honest in range(len(honest_grads_nparray_REDUCED)):

  honest_grads_one_epoch = []
  for gradient_honest in range(CUT_GRAD):
      honest_grads_one_epoch.append(honest_grads_nparray_REDUCED[epoch_honest][gradient_honest])

  honest_grads_reduced_data_rate.append(honest_grads_one_epoch)

print("Number of honest epochs after reduced data rate: {}".format(len(honest_grads_reduced_data_rate)))

# check number of gradients and rates for first 10 epoch
print("First 10 epoch number of gradients and data rates:")
for i in range(10):
  print("({}, {}%) ".format(
    len(honest_grads_reduced_data_rate[i]),
    round((len(honest_grads_reduced_data_rate[i])/len(honest_grads_nparray_REDUCED[i])),2)*100),
    end= " ")

Cut gradients at: 7
Number of honest epochs after reduced data rate: 100
First 10 epoch number of gradients and data rates:
(7, 1.0%)  (7, 1.0%)  (7, 1.0%)  (7, 1.0%)  (7, 1.0%)  (7, 1.0%)  (7, 1.0%)  (7, 1.0%)  (7, 1.0%)  (7, 1.0%)  

In [ ]:
# print(len(honest_grads_nparray))
# print(len(honest_grads_reduced_data_rate))
# print(len(honest_grads_nparray[0]))

# print(len(honest_grads_reduced_data_rate[0]))
# print(len(honest_grads_nparray[0][0]))
# print(len(honest_grads_reduced_data_rate[0][0]))

In [57]:
# Validation for number of gradients (not mandatory)
count_check = 0
for i in range(len(honest_grads_reduced_data_rate[0])):
    # print(honest_grads_reduced_data_rate[0][0][i], " -- ", honest_grads_nparray[0][0][i])
    if honest_grads_reduced_data_rate[0][i][0] == honest_grads_nparray_REDUCED[0][i][0]:
        count_check +=1
    if honest_grads_reduced_data_rate[0][i][0] != honest_grads_nparray_REDUCED[0][i][0]:
        print("Error!")
print(count_check)

7


### 5) Checking gradient and epoch sizes before training LOF

In [58]:
print("Number of FSHA Epochs: ",len(fsha_grads_nparray_REDUCED))
print("Length of one FSHA Epoch: ",len(fsha_grads_nparray_REDUCED[0]))
print("Length of one FSHA Grad: ",len(fsha_grads_nparray_REDUCED[9][20]))
print()
print("Number of Honest Epochs: ",len(honest_grads_reduced_data_rate))
print("Length of one Honest Epoch: ",len(honest_grads_reduced_data_rate[0]))
print("Length of one Honest Grad: ",len(honest_grads_reduced_data_rate[2][3]))

Number of FSHA Epochs:  100
Length of one FSHA Epoch:  782
Length of one FSHA Grad:  1728

Number of Honest Epochs:  100
Length of one Honest Epoch:  7
Length of one Honest Grad:  1728


### 6) Anomaly Detection using Local Outlier Factor(LOF)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

window_size_list = [1, 10, 20]

print(f"LOF Data Rate: {data_rate_honest_gradients}")

epoch_num_honest = len(honest_grads_nparray)

epoch_num_fsha = len(fsha_grads_nparray_REDUCED)
print(f"Number of honest epochs:{epoch_num_honest} - Number of fsha epochs: {epoch_num_fsha}")

for window_size in window_size_list:
  Total_TP_ind1 = 0
  Total_FP_ind1 = 0
  t_detection_point = 0
  t_detection_list = []

  # get TPR(True Positive Rates) results
  # train LOF with (N)th honest epoch and predict (N)th FSHA epoch
  for epoch_fsha in range(epoch_num_fsha): # for FSHA EPOCHS

    honest_epoch_len = len(honest_grads_reduced_data_rate[epoch_fsha])

    time_lof_training_start = time.time()
    lof = LocalOutlierFactor(n_neighbors = (honest_epoch_len-1), novelty = True)
    # train LOF with new epoch's gradients
    lof_novelty = lof.fit(honest_grads_reduced_data_rate[epoch_fsha])
    time_lof_training_end = time.time()


    # get gradients from epoch and evaluate their anomaly score
    for fsha_grad in range(window_size, len(fsha_grads_nparray_REDUCED[epoch_fsha]) - (window_size - 1)):
      # get gradients in selected window size
      fsha_grads_in_window = fsha_grads_nparray_REDUCED[epoch_fsha][fsha_grad - window_size : fsha_grad]
      # print("i: {} | grad window index: [{}, {}]".format(fsha_grad, fsha_grad - window_size, fsha_grad))      # print window index info

      pred_novelty = lof.predict(fsha_grads_in_window)
      inliers_fsha = pred_novelty.tolist().count(1)
      outliers_fsha = pred_novelty.tolist().count(-1)

      if outliers_fsha > inliers_fsha:
        # print("Epoch: {} | Attack is detected in ({})th gradient.  |  t: {}".format(epoch_fsha, fsha_grad, t_detection_point/len(fsha_grads_nparray[epoch_fsha])))
        Total_TP_ind1 +=1
        t_detection_point += fsha_grad/len(trainloader)
        t_detection_list.append(fsha_grad/len(trainloader))
        break
    # print()

  print("# Window size: ", window_size)
  print("Avr TPR:", Total_TP_ind1/epoch_num_fsha)
  t_avr = round(np.average(t_detection_list),4)
  t_std_dev = np.std(t_detection_list, dtype = np.float32)
  t_std_m_err = round(sem(t_detection_list),4)
  print(f"t avr, t SD, t S.ERR: {t_avr}  -  {round(float(t_std_dev),4)}  -  {t_std_m_err}")
  # print("t avr:", round(np.average(t_detection_list),4))
  # print("t std dev:",round(np.std(t_detection_list, dtype = np.float32),4))
  # print("t std err:",round(sem(t_detection_list),4))
  print("-"*35)
  


  # # # get FPR(False Positive Rates) results
  for epoch_honest in range(epoch_num_honest):

    honest_epoch_len = len(honest_grads_reduced_data_rate[epoch_honest])

    time_lof_training_start = time.time()
    lof = LocalOutlierFactor(n_neighbors = (honest_epoch_len-1), novelty = True)
    # train LOF with new epoch's gradients
    lof_novelty = lof.fit(honest_grads_reduced_data_rate[epoch_honest])
    time_lof_training_end = time.time()

    # print(f"# neighbors: {honest_epoch_len-1}, "+
    #     f"  Trained Honest Epoch[{epoch_honest}]: {honest_epoch_len},"+
    #     f"  Tested Honest Epoch[{epoch_honest+1}]: {len(honest_grads_reduced_data_rate[epoch_honest+1])}")

    for honest_grad in range(window_size, len(honest_grads_nparray[epoch_honest]) - (window_size - 1)):

      # get gradients in selected window size
      honest_grads_in_window = honest_grads_nparray[epoch_honest][honest_grad - window_size : honest_grad]

      pred_novelty_honest_test = lof.predict(honest_grads_in_window)
      inliers_honest_test = pred_novelty_honest_test.tolist().count(1)
      outliers_honest_test = pred_novelty_honest_test.tolist().count(-1)

      if outliers_honest_test > inliers_honest_test:
        # print("**Honest Epoch: {} | Attack is detected in ({})th honest gradient.".format(epoch_honest, honest_grad))
        Total_FP_ind1 +=1
        break
    # print()

  print("Window size: ", window_size)
  print("Avr FPR:", Total_FP_ind1/epoch_num_honest)
  print("-"*100)